In [1]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
#!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1
!pip install transformers[torch] accelerate -U


Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Successfully uninstalled tensorflow-2.15.0
tokenizers                       0.19.1
transformers                     4.41.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 7.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidi

In [2]:
%%time
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

paths = "./CorpusUnidosQU.txt"

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

Exception: No such file or directory (os error 2)

In [42]:
#Descargar archivos de google Drive
import gdown
import os
# Lista de archivos necesarios
files = [
    ("16SkLOsfja22kIwExs4NiU5pjrOV7SUdP", "pytorch_model.bin"),
    ("1PrM9LMJ9Pmrc8yqKBT1OMRPXD1urkJ1r", "merges.txt"),
    ("1i6L13u5P9HVzzmKsNZxe_wICteulIWY5", "vocab.json"),
    ("1lDaVeJc90TKbBrhxZKZbIfRTPv9VSsOg", "config.json")
]
folder_name = "quechuaBERT"
os.makedirs(folder_name, exist_ok=True)

# Descargar
for file_id, file_name in files:
    destination_path = os.path.join(folder_name, file_name)
    gdown.download(f"https://drive.google.com/uc?id={file_id}", destination_path, quiet=False)
#tokenizer.save_model("quechuaBERT")

Downloading...
From (original): https://drive.google.com/uc?id=16SkLOsfja22kIwExs4NiU5pjrOV7SUdP
From (redirected): https://drive.google.com/uc?id=16SkLOsfja22kIwExs4NiU5pjrOV7SUdP&confirm=t&uuid=2125564b-1308-4f1e-bae1-5fd341476849
To: /content/quechuaBERT/pytorch_model.bin
100%|██████████| 334M/334M [00:01<00:00, 266MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PrM9LMJ9Pmrc8yqKBT1OMRPXD1urkJ1r
To: /content/quechuaBERT/merges.txt
100%|██████████| 483k/483k [00:00<00:00, 99.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1i6L13u5P9HVzzmKsNZxe_wICteulIWY5
To: /content/quechuaBERT/vocab.json
100%|██████████| 837k/837k [00:00<00:00, 137MB/s]
Downloading...
From: https://drive.google.com/uc?id=1lDaVeJc90TKbBrhxZKZbIfRTPv9VSsOg
To: /content/quechuaBERT/config.json
100%|██████████| 676/676 [00:00<00:00, 1.33MB/s]


In [3]:
#Descargar el corpus de nuestro repositorio https://github.com/Xnehil/TACC-Lexemas/blob/main/data/corpus/corpus.csv
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Xnehil/TACC-Lexemas/main/data/corpus/corpus.csv")
df.head()

#Y guardar un txt; una fila por 'sentence' del dataframe
with open('corpus.txt', 'w') as f:
    for sentence in df['sentence']:
        f.write(sentence + '\n')

In [4]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./quechuaBERT/vocab.json",
    "./quechuaBERT/merges.txt",
)

In [5]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [18]:
tokenizer.encode("allinllachu manan allinlla huk wasipita").tokens

['<s>',
 'allin',
 'llachu',
 'Ġmanan',
 'Ġallinlla',
 'Ġhuk',
 'Ġwasipi',
 'ta',
 '</s>']

In [6]:
# Check that we have a GPU
!nvidia-smi

Sun Jun 16 22:26:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [8]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [9]:
import json
tokenizer_config = {"max_len": 512}

with open("./quechuaBERT/tokenizer_config.json", 'w') as fp:
    json.dump(tokenizer_config, fp)

In [45]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./quechuaBERT", max_len=512)

In [44]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM.from_pretrained("./quechuaBERT")

In [46]:
model.num_parameters()
# => 84 million parameters

83504416

In [47]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./corpus.txt",
    block_size=128,
)

CPU times: user 3.43 s, sys: 32.1 ms, total: 3.46 s
Wall time: 2.17 s


In [48]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [14]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./quechuaBERT",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [15]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss


CPU times: user 5min 54s, sys: 4.58 s, total: 5min 59s
Wall time: 6min 18s


TrainOutput(global_step=333, training_loss=7.987180784299925, metrics={'train_runtime': 378.7887, 'train_samples_per_second': 56.11, 'train_steps_per_second': 0.879, 'total_flos': 619702367651328.0, 'train_loss': 7.987180784299925, 'epoch': 1.0})

In [17]:
trainer.state.log_history[0]

{'train_runtime': 378.7887,
 'train_samples_per_second': 56.11,
 'train_steps_per_second': 0.879,
 'total_flos': 619702367651328.0,
 'train_loss': 7.987180784299925,
 'epoch': 1.0,
 'step': 333}

In [19]:
trainer.save_model("")

In [49]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer="./quechuaBERT"
)

In [50]:
fill_mask("allinllachu <mask> allinlla huk wasipita.")

[{'score': 0.23992183804512024,
  'token': 334,
  'token_str': ' nisqaqa',
  'sequence': 'allinllachu nisqaqa allinlla huk wasipita.'},
 {'score': 0.061005160212516785,
  'token': 16,
  'token_str': ',',
  'sequence': 'allinllachu, allinlla huk wasipita.'},
 {'score': 0.028719963505864143,
  'token': 11,
  'token_str': "'",
  'sequence': "allinllachu' allinlla huk wasipita."},
 {'score': 0.012927922420203686,
  'token': 377,
  'token_str': ' kay',
  'sequence': 'allinllachu kay allinlla huk wasipita.'},
 {'score': 0.012300901114940643,
  'token': 18,
  'token_str': '.',
  'sequence': 'allinllachu. allinlla huk wasipita.'}]

In [41]:
#Runap wiñarquypa puriyninmantam rikuchin.
#Presenta el grado del desarrollo humano .
#<mask>
fill_mask("<mask> wiñarquypa puriyninmantam rikuchin.")

[{'score': 0.03649432212114334,
  'token': 920,
  'token_str': 'Chay',
  'sequence': 'Chay wiñarquypa puriyninmantam rikuchin.'},
 {'score': 0.027767308056354523,
  'token': 2953,
  'token_str': '¿',
  'sequence': '¿ wiñarquypa puriyninmantam rikuchin.'},
 {'score': 0.01818837597966194,
  'token': 664,
  'token_str': 'Kay',
  'sequence': 'Kay wiñarquypa puriyninmantam rikuchin.'},
 {'score': 0.004317810758948326,
  'token': 2031,
  'token_str': 'Ima',
  'sequence': 'Ima wiñarquypa puriyninmantam rikuchin.'},
 {'score': 0.004133380018174648,
  'token': 18,
  'token_str': '.',
  'sequence': '. wiñarquypa puriyninmantam rikuchin.'}]

In [ ]:
# Conexion a Google Colaborative
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive
